Let's see how the XHMM results change as we vary the quality threshold. First, classify CNVs between denovo and inherited using PSEQ:

In [2]:
module load plinkseq
module load plink/1.07

cd ~/data/cnv/xhmm

for q in 50 60 70 80 90; do
    pseq DATA cnv-denovo --noweb --minSQ $q --minNQ $q --out DATA_q${q}
    grep DENOVO DATA_q${q}.denovo.cnv > pseq_DENOVO.txt
    # borrow the header row
    head -1 DATA.xcnv > denovo.xcnv;

    # filter out denovo CNVs
    while read sample; do
        grep $sample DATA.xcnv > sample.xcnv;
        for cnv in `grep $sample pseq_DENOVO.txt | cut -f 3 -`; do
            # replacing .. by -
            cnv=`echo $cnv | sed -e 's/\.\./\-/'`;
            grep $cnv sample.xcnv >> denovo.xcnv; 
        done;
    done < ../kid_samples.txt;
    /usr/local/apps/XHMM/2016-01-04/sources/scripts/xcnv_to_cnv denovo.xcnv > tmp.cnv
    # switch around FAMID and IID columns, and remove header
    awk '{OFS="\t"; if ( $3 != "CHR" ) {print $2, $1, $3, $4, $5, $6, $7, $8 }}' tmp.cnv > denovo_q${q}.cnv
    rm sample.xcnv pseq_DENOVO.txt tmp.cnv denovo.xcnv
    
    # filter out inherited cnvs
    grep MATERNAL_TRANSMITTED DATA_q${q}.denovo.cnv > pseq_TRANSMITTED.txt
    grep PATERNAL_TRANSMITTED DATA_q${q}.denovo.cnv >> pseq_TRANSMITTED.txt
    # borrow the header row
    head -1 DATA.xcnv > inherited.xcnv;

    while read sample; do
        grep $sample DATA.xcnv > sample.xcnv;
        for cnv in `grep $sample pseq_TRANSMITTED.txt | cut -f 3 -`; do
            # replacing .. by -
            cnv=`echo $cnv | sed -e 's/\.\./\-/'`;
            grep $cnv sample.xcnv >> inherited.xcnv; 
        done;
    done < ../kid_samples.txt;
    /usr/local/apps/XHMM/2016-01-04/sources/scripts/xcnv_to_cnv inherited.xcnv > tmp.cnv
    # switch around FAMID and IID columns, and remove header
    awk '{OFS="\t"; if ( $3 != "CHR" ) {print $2, $1, $3, $4, $5, $6, $7, $8 }}' tmp.cnv > inherited_q${q}.cnv
    rm sample.xcnv pseq_TRANSMITTED.txt tmp.cnv inherited.xcnv
    
    # compile all CNVs for kids
    # borrow the header row
    head -1 DATA.xcnv > all.xcnv;

    # effectively just filtering DATA.xcnv to keep only kids
    while read sample; do
        grep $sample DATA.xcnv >> all.xcnv;
    done < ../kid_samples.txt;
    /usr/local/apps/XHMM/2016-01-04/sources/scripts/xcnv_to_cnv all.xcnv > tmp.cnv
    # switch around FAMID and IID columns, and remove header
    awk '{OFS="\t"; if ( $3 != "CHR" ) {print $2, $1, $3, $4, $5, $6, $7, $8 }}' tmp.cnv > all_q${q}.cnv
    rm tmp.cnv all.xcnv
done

[-] Unloading GSL 2.2.1 ...
[-] Unloading Graphviz v2.38.0 ...
[-] Unloading gdal 2.0 ...
[-] Unloading proj 4.9.2 ...
[-] Unloading gcc 4.9.1 ...
[-] Unloading openmpi 1.10.0 for GCC 4.9.1
[-] Unloading tcl_tk 8.6.3
[-] Unloading Zlib 1.2.8 ...
[-] Unloading Bzip2 1.0.6 ...
[-] Unloading pcre 8.38 ...
[-] Unloading liblzma 5.2.2 ...
[-] Unloading libjpeg-turbo 1.5.1 ...
[-] Unloading tiff 4.0.7 ...
[-] Unloading curl 7.46.0 ...
[-] Unloading boost libraries v1.65 ...
[-] Unloading R 3.4.0 on cn3238
[+] Loading GSL 2.2.1 ...
[+] Loading Graphviz v2.38.0 ...
[+] Loading gdal 2.0 ...
[+] Loading proj 4.9.2 ...
[+] Loading gcc 4.9.1 ...
[+] Loading openmpi 1.10.0 for GCC 4.9.1
[+] Loading tcl_tk 8.6.3
[+] Loading Zlib 1.2.8 ...
[+] Loading Bzip2 1.0.6 ...
[+] Loading pcre 8.38 ...
[+] Loading liblzma 5.2.2 ...
[-] Unloading Zlib 1.2.8 ...
[+] Loading Zlib 1.2.8 ...
[-] Unloading liblzma 5.2.2 ...
[+] Loading liblzma 5.2.2 ...
[+] Loading libjpeg-turbo 1.5.1 ...
[+] Loading tiff 4.0.7 ...


Now we create all necessary PLINK files:

In [ ]:
for q in 50 60 70 80 90; do
    for cnvtype in all denovo inherited; do
        cnvname=${cnvtype}_q${q}.cnv
        plink --cnv-list $cnvname --cnv-make-map --noweb --out ${cnvtype}_q${q};
        
        # remove bad regions
        plink --map ${cnvname}.map --fam ../simplex_nofamid.ped --cnv-list $cnvname \
            --noweb --1 --cnv-exclude bad_regions.list --cnv-overlap .5 \
            --cnv-write --out ${cnvtype}_q${q}_clean
        plink --cnv-list ${cnvtype}_q${q}_clean.cnv --cnv-make-map --noweb --1 \
            --out ${cnvtype}_q${q}_clean
        
        for qc in '' '_clean'; do
            cnvname=${cnvtype}_q${q}${qc}.cnv
            # whole burden
            plink --map ${cnvname}.map --fam ../simplex_nofamid.ped --cnv-list $cnvname \
                --noweb --1 --cnv-check-no-overlap --out ${cnvtype}_q${q}${qc}_burden;
            # gene sets
            plink --map ${cnvname}.map --fam ../simplex_nofamid.ped --cnv-list $cnvname \
                --noweb --1 --cnv-intersect glist-hg19 --cnv-verbose-report-regions \
                --cnv-subset genes.txt --out ${cnvtype}_q${q}${qc}_genes;
            plink --map ${cnvname}.map --fam ../simplex_nofamid.ped --cnv-list $cnvname \
                --noweb --1 --cnv-intersect glist-hg19 --cnv-verbose-report-regions \
                --cnv-subset hg19_allenBrainGene_trimmed.txt \
                --out ${cnvtype}_q${q}${qc}_brainGenes;
            # subtypes only
            for sub in del dup; do
                plink --map ${cnvname}.map --fam ../simplex_nofamid.ped --cnv-list $cnvname \
                --noweb --1 --cnv-${sub} --out ${cnvtype}_q${q}${qc}_${sub}Burden;
                # gene sets
                plink --map ${cnvname}.map --fam ../simplex_nofamid.ped --cnv-list $cnvname \
                    --noweb --1 --cnv-intersect glist-hg19 --cnv-verbose-report-regions \
                    --cnv-subset genes.txt --cnv-${sub} \
                    --out ${cnvtype}_q${q}${qc}_${sub}Genes;
                plink --map ${cnvname}.map --fam ../simplex_nofamid.ped --cnv-list $cnvname \
                    --noweb --1 --cnv-intersect glist-hg19 --cnv-verbose-report-regions \
                    --cnv-subset hg19_allenBrainGene_trimmed.txt --cnv-${sub} \
                    --out ${cnvtype}_q${q}${qc}_${sub}BrainGenes;
            done;
        done;
    done;
done


@----------------------------------------------------------@
|        PLINK!       |     v1.07      |   10/Aug/2009     |
|----------------------------------------------------------|
|  (C) 2009 Shaun Purcell, GNU General Public License, v2  |
|----------------------------------------------------------|
|  For documentation, citation & bug-report instructions:  |
|        http://pngu.mgh.harvard.edu/purcell/plink/        |
@----------------------------------------------------------@

Skipping web check... [ --noweb ] 
Writing this text to log file [ all_q50.log ]
Analysis started: Wed Jan  3 17:03:25 2018

Options in effect:
	--cnv-list all_q50.cnv
	--cnv-make-map
	--noweb
	--out all_q50


Reading segment list (CNVs) from [ all_q50.cnv ]
Writing new MAP file to [ all_q50.cnv.map ]
Wrote 10414 unique positions to file

Analysis finished: Wed Jan  3 17:03:25 2018


@----------------------------------------------------------@
|        PLINK!       |     v1.07      |   10/Aug/2009     |
|